# Group 1 ETL Project

### Introduction...

In [1]:
import pandas as pd
import numpy as np
import os

# Crime data in Mexico City

You can download the data in the next link: https://drive.google.com/file/d/1uD9NdAsV3tlXhqvv6WEpV5nR8gqDjsN7/view

In [ ]:
#path=os.path.join("Resources","IDM_NM_feb2021.csv")
#crime=pd.read_csv(path,encoding="Latin-1")

In [ ]:
crime.head(3)

In [ ]:
crime['Tipo de delito'].unique()

In [ ]:
crime=crime[crime['Tipo de delito']=="Robo"]
crime['Subtipo de delito'].unique()

In [ ]:
cols=['Enero',
 'Febrero',
 'Marzo',
 'Abril',
 'Mayo',
 'Junio',
 'Julio',
 'Agosto',
 'Septiembre',
 'Octubre',
 'Noviembre',
 'Diciembre']

In [ ]:
list(crime)

In [ ]:
crime["tot_robs"]=crime[cols].sum(axis=1)

In [ ]:
#we want only CDMX which code is 9
cl1=(crime['Clave_Ent']==9)
#We want 2020 to match COVID and CENSUS
cl2=(crime["Año"]<2021)

crime=crime[cl1&cl2]

In [ ]:
#Collecting only the columns we are interested in
cols=['Año',
 'Cve. Municipio',
 'Municipio',
 'Modalidad',
 'tot_robs']

crime=crime[cols]

crime.rename(columns={"Año":"year","Cve. Municipio":"cmun","Municipio":"mun","Modalidad":'modality'},inplace=True)

In [ ]:
crime=crime[crime.cmun<9016]

In [ ]:
crime=crime.groupby(['year','cmun','mun','modality'],as_index=False).sum()

In [ ]:
crime.head()

In [ ]:
path=os.path.join("Resources","crime.csv")
crime.to_csv(path,index=False)

In [13]:
crime_df.head()

,year,cmun,mun,modality,tot_robs
0,2015,9002,Azcapotzalco,Con violencia,883.0
1,2015,9002,Azcapotzalco,"Robo de cables, tubos y otros objetos destinad...",0.0
2,2015,9002,Azcapotzalco,"Robo de cables, tubos y otros objetos destinad...",0.0
3,2015,9002,Azcapotzalco,Robo de coche de 4 ruedas Con violencia,159.0
4,2015,9002,Azcapotzalco,Robo de coche de 4 ruedas Sin violencia,559.0


# Population data in Mexico City

In [ ]:
#path=os.path.join("Resources","ITER_NALCSV20.csv")
#pop=pd.read_csv(path,encoding="Latin-1")

You can download the data in the next link: https://www.inegi.org.mx/programas/ccpv/2020/#Datos_abiertos

In [ ]:
pop.tail(2)

In [ ]:
# We only want pop in Mexico City, code 9
pop=pop[pop['ï»¿ENTIDAD']==9]
# We kept only 
pop=pop[['ï»¿ENTIDAD','MUN','POBTOT']]

pop.rename(columns={'ï»¿ENTIDAD':'state','MUN':'mun','POBTOT':'totpop'},inplace=True)

In [ ]:
pop=pop[pop['mun']>0]
pop['cmun']=pop.state*1000+pop.mun
pop=pop[['cmun','totpop']].groupby('cmun',as_index=False).sum()

In [ ]:
pop['year']=2020

In [ ]:
pop

In [ ]:
path=os.path.join("Resources","population.csv")
pop.to_csv(path,index=False)

In [17]:
population_df.head()

,cmun,totpop,year
0,9002,864410,2020
1,9003,1228894,2020
2,9004,435415,2020
3,9005,2346702,2020
4,9006,809390,2020


# COVID cases in Mexico City

In [ ]:
#path=os.path.join("Resources","sisver_public.csv")
#covid=pd.read_csv(path,encoding="Latin-1")

The data can be downladed in the next link: https://datos.cdmx.gob.mx/dataset/base-covid-sinave

In [ ]:
covid=covid[covid['cventine']==9]

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
covid['year']=[i[:4] for i in covid['fechreg']]

In [ ]:
covid=covid[['year','cventine','cvemuni','tipacien','evoluci','id']]

In [ ]:
covid=covid.groupby(['year','cventine','cvemuni','tipacien','evoluci'],as_index=False).count()

In [ ]:
covid['cmun']=covid['cventine']*1000+covid['cvemuni']

In [ ]:
covid.head()

In [ ]:
path=os.path.join("Resources","covid.csv")
covid.to_csv(path,index=False)

## Load stage

In [2]:
# Convert transformed csv files into dataframes

path_crime=os.path.join("Resources","crime.csv")
crime_df=pd.read_csv(path_crime,encoding="Latin-1")

path_population=os.path.join("Resources","population.csv")
population_df=pd.read_csv(path_population,encoding="Latin-1")

path_covid=os.path.join("Resources","covid.csv")
covid_df=pd.read_csv(path_covid,encoding="Latin-1")


### Connect to database

In [9]:
 
from sqlalchemy import create_engine
rds_connection_string = "postgres:postgres@localhost:5432/ETL Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [11]:
!pip install psycopg2-binary

     |████████████████████████████████| 1.5 MB 1.4 MB/s eta 0:00:01


### Load into database

In [21]:
crime_df.to_sql(name='crime', con=engine, if_exists='append', index=False)
population_df.to_sql(name='population', con=engine, if_exists='append', index=False)  
covid_df.to_sql(name='covid', con=engine, if_exists='append', index=False)  

### Query in database

In [22]:
 pd.read_sql_query('select * from crime', con=engine).head()

,id,year,cmun,mun,modality,tot_robs
0,1,2015,9002,Azcapotzalco,Con violencia,883
1,2,2015,9002,Azcapotzalco,"Robo de cables, tubos y otros objetos destinad...",0
2,3,2015,9002,Azcapotzalco,"Robo de cables, tubos y otros objetos destinad...",0
3,4,2015,9002,Azcapotzalco,Robo de coche de 4 ruedas Con violencia,159
4,5,2015,9002,Azcapotzalco,Robo de coche de 4 ruedas Sin violencia,559
